In [2]:
import langchain
from langchain_community.document_loaders import PyPDFLoader


In [3]:

def load_pdf(file_path):
    """Loads a PDF and extracts text from it."""
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

pdf_path = "../ikigai.pdf"
documents = load_pdf(pdf_path)


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_text(documents, chunk_size=500, chunk_overlap=100):
    """Splits extracted text into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

text_chunks = chunk_text(documents)

for i, chunk in enumerate(text_chunks[:5]):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")


Chunk 1:
IKIGAI 
1. What is your reason for being? 
2. Whatever you do, don’t retire! 
3. Having a clearly defined ikigai brings satisfaction, 
happiness, and meaning to our lives. 
4. Their blood tests reveal fewer free radicals (which are 
responsible for cellular aging), as a result of drinking tea 
and eating until their stomachs are only 80 percent full 
5. the keys to longevity are diet, exercise, finding a purpose 
in life (an ikigai), and forming strong social ties—that is,

Chunk 2:
in life (an ikigai), and forming strong social ties—that is, 
having a broad circle of friends and good family relations. 
6. Members of these communities manage their time well in 
order to reduce 
7. stress, consume little meat or processed foods, and drink 
alcohol in moderation.1 They don’t do strenuous exercise, 
but they do move every day, taking walks and working in 
their vegetable gardens 
8. Fill your belly to 80 percent.” Ancient wisdom advises

Chunk 3:
their vegetable gardens 
8. Fill 

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
import google.generativeai as genai
import faiss
import numpy as np
from langchain_google_genai import GoogleGenerativeAIEmbeddings


# Initialize Gemini Embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [10]:
def generate_gemini_embeddings(text_chunks):
    """Converts text chunks into Gemini embeddings."""
    texts = [chunk.page_content for chunk in text_chunks]
    embeddings = embeddings_model.embed_documents(texts)
    return np.array(embeddings).astype("float32"), texts

# Convert text chunks to embeddings
embeddings, texts = generate_gemini_embeddings(text_chunks)


In [11]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, "faiss.index")


In [16]:
# Save text chunks separately
with open("stored_chunks.pkl", "wb") as f:
    pickle.dump(texts, f)  # `texts` contains the original text chunks


In [17]:
import faiss
import pickle

# Load FAISS index
index = faiss.read_index("faiss.index")

# Load stored text chunks (stored separately in a pickle file)
with open("stored_chunks.pkl", "rb") as f:
    texts = pickle.load(f)  # This should contain the chunked text


In [36]:
def extract_topics_from_chunks_and_summarize(chunks):
    """Extract key topics from stored FAISS chunks."""
    combined_text = "\n".join(chunks)  # Merge chunks for topic extraction
    model = genai.GenerativeModel('gemini-pro')
    prompt=f"Extract the most important topics from the following study material :\n\n{combined_text}"

    response = model.generate_content(prompt)
    print(response.text.strip())
    

    # topics = response.text.split()  # Convert response to list
    # #topics = topics.replace("'''", "").replace('"""', "").replace("JSON", "").strip()
    prompt=f"Summarize the following and just return the summary:\n\n{combined_text}"
    response = model.generate_content(prompt)
    
    print(response.text.strip())

# Extract topics from the stored chunks
extract_topics_from_chunks_and_summarize(texts)
# important_topics = extract_topics_from_chunks(texts)
# print("\nExtracted Topics:\n", important_topics)


**Key Topics:**

**Ikigai:**

* A Japanese concept referring to one's reason for being.
* Brings fulfillment, happiness, and meaning to life.
* Four components: what you love, what you're good at, what the world needs, and what you can be rewarded for.

**Longevity:**

* Factors contributing to long life include diet, exercise, purpose, and social connections.
* The Okinawa centenarians serve as examples of healthy and long-lived populations.
* Key principles: reduce stress, eat a balanced diet, maintain an active lifestyle, and avoid excessive consumption.

**Mindfulness and Flow:**

* Mindfulness helps reduce stress and anxiety.
* Flow is a state of complete immersion and enjoyment in an activity.
* Strategies for achieving flow include setting clear goals, practicing concentration, avoiding distractions, and pursuing activities that align with one's passions.

**Resilience and Antifragility:**

* The ability to withstand adversity and bounce back is crucial for a meaningful life.
* 

In [35]:
def gen_quiz(chunks):
    """Extract key topics from stored FAISS chunks."""
    combined_text = "\n".join(chunks)  # Merge chunks for topic extraction
    model = genai.GenerativeModel('gemini-pro')

    

    # topics = response.text.split()  # Convert response to list
    # #topics = topics.replace("'''", "").replace('"""', "").replace("JSON", "").strip()
    prompt=f"Generate Quiz for the following content: \n\n{combined_text}"
    response = model.generate_content(prompt)
    
    print(response.text.strip())

# Extract topics from the stored chunks
gen_quiz(texts)
# important_topics = extract_topics_from_chunks(texts)
# print("\nExtracted Topics:\n", important_topics)


**Quiz**

**Instructions:** Answer the following questions based on the provided content on ikigai.

1. What does ikigai represent?
2. Should you retire from work when you find your ikigai?
3. What does having a clearly defined ikigai bring to our lives?
4. How do tea and eating until you're 80% full affect longevity?
5. List the five keys to longevity according to the content.
6. What do members of longevity communities do to reduce stress?
7. According to the "Hara hachi bu" saying, what should you do when eating?
8. How can presenting meals on many small plates help control food intake?
9. Define "moai."
10. What are the perceived benefits of being part of a moai?
11. Translate "mens sana in corpore sano" and explain its significance.
12. Why is it important to expose yourself to change?
13. What is the central premise of the stress-reduction method presented in the content?
14. How does high stress affect cells?
15. Should we eliminate high-stress lifestyles entirely?
16. What are 